# [Sec0] geting Data from postgresql

## ArticleScoreTable Example

In [1]:
import psycopg2
#connect to pg
conn = psycopg2.connect("dbname='foodmining' user='' host='' password=''")
import pandas.io.sql as psql

In [2]:
dataframe = psql.read_sql("SELECT * FROM ArticleScoreTable", conn)
dataframe.shape

(413, 8)

In [3]:
dataframe.iloc[:,2:].sum(axis=0)

ch       172
jp        64
ko        57
tai       23
west      93
other     31
dtype: int64

In [4]:
dataframe_article = psql.read_sql("SELECT * FROM ArticleTable LIMIT 5", conn)
dataframe_article.shape

(5, 7)

In [5]:
print (dataframe_article.tail())

   articleid            title    author  \
0          1         [吃] 廿二碼頭       MBM   
1          2         [吃] 石記魚丸       MBM   
2          3       [推薦]山月亭養生鍋      yahe   
3          4  Re: [問題] 敦化國中附近     pchan   
4          5      [心得] 蘿拉義大利麵  snowbabe   

                                                 url  \
0  https://www.ptt.cc/bbs/Food/M.1063740976.A.260...   
1  https://www.ptt.cc/bbs/Food/M.1065848540.A.68C...   
2  https://www.ptt.cc/bbs/Food/M.1075957265.A.BB6...   
3  https://www.ptt.cc/bbs/Food/M.1076404948.A.EAA...   
4  https://www.ptt.cc/bbs/Food/M.1077161786.A.24F...   

                       date  \
0  Wed Sep 17 03:36:27 2003   
1  Sat Oct 11 13:02:27 2003   
2  Thu Feb  5 13:22:07 2004   
3  Tue Feb 10 17:33:33 2004   
4  Thu Feb 19 12:09:24 2004   

                                             context                  latlon  
0  作者MBM (餅)看板Food標題[吃] 廿二碼頭  時間Wed Sep 17 03:36:...    25.170909,121.438352  
1  作者MBM (餅)看板Food標題[吃] 石記魚丸時間Sat Oct 11 13:02:27...  24.79

## StoreTable Example

In [6]:
dataframe = psql.read_sql("SELECT * FROM StoreTable LIMIT 5", conn)
dataframe.shape

(5, 5)

In [7]:
print (dataframe.tail())

   storeid                         address              storename  \
0        1          台北縣淡水鎮中正路11巷22號 (渡船頭邊)                   None   
1        3                        基隆是愛六路九號                   None   
2        4  台北市南京東路三段 256巷41號  電話2752-0939  對不起沒有拿名片 不記得也不清楚詳細的地址   
3        5    : 台北市延吉街131巷8號  (就是原本薰衣草的位置)                   None   
4        6  我承認循著找過去...第一眼我好失望..因為雖然是新開的..                   None   

                   latlon  count  
0    25.170909,121.438352      1  
1  25.1277141,121.7472756      1  
2    25.0498683,121.54459      1  
3   25.0425802,121.554721      1  
4   38.9176881,121.614627      1  


In [8]:
dataframe = psql.read_sql("SELECT * FROM PushTable LIMIT 5", conn)
dataframe.shape

(5, 5)

In [9]:
print (dataframe.tail())

   pushid                                                url     userid  \
0       1  https://www.ptt.cc/bbs/Food/M.1079380714.A.108...     abainn   
1       2  https://www.ptt.cc/bbs/Food/M.1084581681.A.E5A...  HolyStaff   
2       3  https://www.ptt.cc/bbs/Food/M.1089710853.A.C8B...       hazy   
3       4  https://www.ptt.cc/bbs/Food/M.1089710853.A.C8B...     schuey   
4       5  https://www.ptt.cc/bbs/Food/M.1091374670.A.A36...     dk3y93   

  pushtag           pushcontext  
0      →                 問有電話嗎？  
1      推                   推綠左大  
2      推          一家富美跟南方澳的一樣嘛?  
3      推                    一樣~  
4      推   家口味並不是很道地/在我家附近而已@@"  


## [Sec1]  Let's perform chinese word split with jieba
installation:
```
pip install jieba
```

In [10]:
import jieba

In [11]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.858 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学


In [12]:
seg_list = jieba.cut("我来到北京清华大学")
[i for i in seg_list] # way to extract terms from generator.

['我', '来到', '北京', '清华大学']

In [13]:
import re

def remove_punct(x,debug=False):
    if debug:
        print(x)
    return re.sub('[：！\[\]:]',' ',x)

remove_punct(dataframe_article.title[0],debug=True)

[吃] 廿二碼頭


' 吃  廿二碼頭'

In [14]:
seg_list = jieba.cut(remove_punct(dataframe_article.title[0]))
[i for i in seg_list] # way to extract terms from generator.

[' ', '吃', ' ', ' ', '廿二', '碼頭']

## [Sec2] Create DTM by using Scikit Learn :CountVectorizer

there are also sklearn.feature_extraction.text.TfidfVectorizer


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer(min_df=1)


X = vectorizer.fit_transform(corpus)
print (vectorizer.get_feature_names())
print (X.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


In [16]:
#try on chinese
corpus = [
     '莫 你 是在 靠北',
     '靠北 交大 有 三篇 廢文',
     '靠北 是 世界奇觀 帥',
     '交大 林駿羽 帥 帥 帥',
]
vectorizer = CountVectorizer(min_df=0,token_pattern=u'\w+')


X = vectorizer.fit_transform(corpus)
print (vectorizer.get_feature_names())
print (X.toarray())


['三篇', '世界奇觀', '交大', '你', '帥', '廢文', '是', '是在', '有', '林駿羽', '莫', '靠北']
[[0 0 0 1 0 0 0 1 0 0 1 1]
 [1 0 1 0 0 1 0 0 1 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 0 1]
 [0 0 1 0 3 0 0 0 0 1 0 0]]


In [17]:
# create pandas matrix from above
import pandas as pd
df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
# this seems not difficult at all
df

,三篇,世界奇觀,交大,你,帥,廢文,是,是在,有,林駿羽,莫,靠北
0,0,0,0,1,0,0,0,1,0,0,1,1
1,1,0,1,0,0,1,0,0,1,0,0,1
2,0,1,0,0,1,0,1,0,0,0,0,1
3,0,0,1,0,3,0,0,0,0,1,0,0


In [18]:
# apply the techique on our data set
seg_list = jieba.cut(remove_punct(dataframe_article.title[0])) # split
term_list = [i for i in seg_list if i != ' '] # way to extract terms from generator.
term_string = ' '.join(term_list) # concate terms with a space
print(term_string) 

吃 廿二 碼頭


In [19]:
# rewrite code as function
def Chinese_splitter(x, output_list=False):
    seg_list = jieba.cut(remove_punct(x)) # split
    term_list = [i for i in seg_list if i != ' '] # way to extract terms from generator.
    if output_list:
        return term_list
    term_string = ' '.join(term_list) # concate terms with a space
    return term_string

In [20]:
corpus = [Chinese_splitter(i) for i in dataframe_article.title]
print(corpus) # 義大利怪怪der

['吃 廿二 碼頭', '吃 石記 魚丸', '推薦 山月亭 養生鍋', 'Re 問題 敦化 國中 附近', '心得 蘿拉義 大利 麵']


In [22]:
# 好像切的怪怪的, 找一下繁體詞庫 （義大利）
jieba.set_dictionary('dict.txt.big')
corpus = [Chinese_splitter(i) for i in dataframe_article.title]
print(corpus)
# 喔喔喔喔喔！！

Building prefix dict from /home/stream/Downloads/ChJpKoThailWest/dict.txt.big ...
Loading model from cache /tmp/jieba.u5c6697f267681021cfc5792381b66031.cache
Loading model cost 1.402 seconds.
Prefix dict has been built succesfully.


['吃 廿二 碼頭', '吃 石記 魚丸', '推薦 山月亭 養生 鍋', 'Re 問題 敦化國中 附近', '心得 蘿拉 義大利 麵']


In [24]:
vectorizer = CountVectorizer(min_df=0,token_pattern=u'\w+')
X = vectorizer.fit_transform(corpus)
df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
# this seems not difficult at all
df

,re,吃,問題,山月亭,廿二,心得,推薦,敦化國中,石記,碼頭,義大利,蘿拉,鍋,附近,養生,魚丸,麵
0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0
3,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1


## [Sec 3] Join AritcleStoreTable and Ariticle Table and Create X , Y

SELECT * FROM table1 INNER JOIN table2 ON table1.common_filed = table2.common_field;

In [28]:
dataframe = psql.read_sql("SELECT * FROM ArticleScoreTable INNER JOIN ArticleTable ON ArticleScoreTable.url = ArticleTable.url", conn)
dataframe.shape

(413, 15)

In [29]:
dataframe # seems good

,url,score,ch,jp,ko,tai,west,other,articleid,title,author,url,date,context,latlon
0,https://www.ptt.cc/bbs/Food/M.1440654584.A.F2F...,1,True,True,False,False,False,False,70470,[食記] 新竹東區關新北路開丼燒肉飯,lovecindy,https://www.ptt.cc/bbs/Food/M.1440654584.A.F2F...,Thu Aug 27 13:49:39 2015,作者lovecindy (lovecindy)看板Food標題[食記] 新竹東區關新北路開丼...,"24.7920604,120.9933678"
1,https://www.ptt.cc/bbs/Food/M.1391700658.A.04C...,1,True,False,False,False,False,False,37987,[食記] 新竹_老張鴨肉麵,somnolence,https://www.ptt.cc/bbs/Food/M.1391700658.A.04C...,Thu Feb 6 23:30:54 2014,作者somnolence (Ku~kuku~)看板Food標題[食記] 新竹_老張鴨肉麵時間...,"24.7992891,120.9732165"
2,https://www.ptt.cc/bbs/Food/M.1446565831.A.71B...,1,False,False,False,False,True,False,73931,[廣宣] 新竹 艾莎異國美食 低調有質感,PennysHouse,https://www.ptt.cc/bbs/Food/M.1446565831.A.71B...,Tue Nov 3 23:50:27 2015,作者PennysHouse (Penny's House)看板Food標題[廣宣] 新竹 艾...,"24.805524,120.982918"
3,https://www.ptt.cc/bbs/Food/M.1426079189.A.6A1...,0,False,False,False,False,False,True,61212,[食記] 新竹_莫尼漢堡（Morning Burger）,somnolence,https://www.ptt.cc/bbs/Food/M.1426079189.A.6A1...,Wed Mar 11 21:06:20 2015,作者somnolence (Ku~kuku~)看板Food標題[食記] 新竹_莫尼漢堡（Mo...,"24.78963,120.9994211"
4,https://www.ptt.cc/bbs/Food/M.1361195490.A.F44...,0,False,False,False,False,False,True,20665,[食記] 新竹。老窩咖啡 Laoo Coffee,jescal,https://www.ptt.cc/bbs/Food/M.1361195490.A.F44...,Mon Feb 18 21:51:28 2013,作者jescal ()看板Food標題[食記] 新竹。老窩咖啡 Laoo Coffee時間M...,"24.7847536,120.9901789"
5,https://www.ptt.cc/bbs/Food/M.1478855632.A.1B9...,1,False,False,False,False,False,True,92850,[食記] 新竹 遙遠高翠路的烘烘甜點店,k272821,https://www.ptt.cc/bbs/Food/M.1478855632.A.1B9...,Fri Nov 11 17:13:50 2016,作者k272821 (cc)看板Food標題[食記] 新竹 遙遠高翠路的烘烘甜點店時間Fri...,"24.7808646,120.9860119"
6,https://www.ptt.cc/bbs/Food/M.1380099916.A.011...,0,True,False,False,False,False,False,30857,[廣宣] 新竹 LoVe茶滷味 位於園區和清大後門,changsh1128,https://www.ptt.cc/bbs/Food/M.1380099916.A.011...,Wed Sep 25 17:05:14 2013,作者changsh1128 (Ivan)看板Food標題[廣宣] 新竹 LoVe茶滷味 ...,"24.7788799,120.9857572"
7,https://www.ptt.cc/bbs/Food/M.1180873775.A.FB0...,1,True,False,False,False,False,False,1661,[食記]新竹--北平半畝園之「刀削炸醬麵」,trulip,https://www.ptt.cc/bbs/Food/M.1180873775.A.FB0...,Sun Jun 3 20:29:34 2007,作者trulip (佛羅倫斯)看板Food標題[食記]新竹--北平半畝園之「刀削炸醬麵」時間...,"24.7787822,120.9860932"
8,https://www.ptt.cc/bbs/Food/M.1392223285.A.4F4...,0,True,False,False,False,False,False,38399,[食記] 新竹。半畝園,jescal,https://www.ptt.cc/bbs/Food/M.1392223285.A.4F4...,Thu Feb 13 00:41:22 2014,作者jescal ()看板Food標題[食記] 新竹。半畝園(二訪)。時間Thu Feb 1...,"24.7784581,120.9863438"
9,https://www.ptt.cc/bbs/Food/M.1402049448.A.5AA...,0,True,False,False,False,False,False,45154,[食記] 新竹-半畝園-園區後面的秘密麵攤,goodian,https://www.ptt.cc/bbs/Food/M.1402049448.A.5AA...,Fri Jun 6 18:10:41 2014,作者goodian (小天)看板Food標題[食記] 新竹-半畝園-園區後面的秘密麵攤時間F...,"24.7784581,120.9863438"


In [39]:
# Create X first
corpus = [Chinese_splitter(i) for i in dataframe.title]
vectorizer = CountVectorizer(min_df=5,token_pattern=u'\w+')
X = vectorizer.fit_transform(corpus)
X = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
X

,2,_,bar,cp,dak,dining,eat,fw,go,jjim,...,食館,飯,飯店,餃子,餐,餐廳,館,高雄,魚,麵
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
print (df.sum(axis=0))
# 看起來有些廢字應該要處理

2          11
_          16
bar         5
cp          5
dak         5
dining      5
eat         8
fw         18
go          8
jjim        5
tapas       9
一中         40
一道          6
不           6
中          83
中山         11
中興新村       12
丼          11
九尾          7
亭           6
值           5
偈           6
兔子         13
創意          9
北區         16
十巷          6
南投         46
台          80
台北         58
吃           5
         ... 
自           6
臭豆腐         7
草屯         26
街          29
西班牙         5
豆腐         20
貳           9
超           5
遊者          5
酒館          5
金元寶         5
鍋           6
鐵板燒         5
長鼻          5
雙魚         12
韓國         19
韓式         15
韓門          7
頂好          7
食記        384
食館          5
飯           7
飯店          6
餃子          7
餐           9
餐廳         23
館          13
高雄         19
魚           5
麵          80
Length: 113, dtype: int64


In [37]:
# 不過現在已經是一個可以 apply ML 的樣子惹, 有 X 有Y
Y = dataframe.ch

## [Sec4] apply xgboost to classify chinese or not

In [112]:
import xgboost as xgb
import numpy as np
import random

In [122]:
msk = np.random.rand(len(X)) < 0.8
dtrain = xgb.DMatrix(X[msk], Y[msk])
dtest = xgb.DMatrix(X[~msk], Y[~msk])

In [124]:
param = {'max_depth':8, 'eta':0.001, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 6
param['eval_metric'] = 'error'
evallist  = [(dtrain,'train'),(dtest,'eval')]
num_round = 8000
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=300)

[0]	train-error:0.386228	eval-error:0.227848
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 300 rounds.
[1]	train-error:0.347305	eval-error:0.21519
[2]	train-error:0.323353	eval-error:0.253165
[3]	train-error:0.314371	eval-error:0.240506
[4]	train-error:0.320359	eval-error:0.227848
[5]	train-error:0.326347	eval-error:0.227848
[6]	train-error:0.335329	eval-error:0.253165
[7]	train-error:0.335329	eval-error:0.253165
[8]	train-error:0.335329	eval-error:0.253165
[9]	train-error:0.335329	eval-error:0.253165
[10]	train-error:0.317365	eval-error:0.21519
[11]	train-error:0.311377	eval-error:0.21519
[12]	train-error:0.323353	eval-error:0.227848
[13]	train-error:0.323353	eval-error:0.227848
[14]	train-error:0.314371	eval-error:0.240506
[15]	train-error:0.311377	eval-error:0.240506
[16]	train-error:0.311377	eval-error:0.240506
[17]	train-error:0.311377	eval-error:0.240506
[18]	train-error:0.311377	eval-error:0.

[174]	train-error:0.302395	eval-error:0.291139
[175]	train-error:0.305389	eval-error:0.291139
[176]	train-error:0.305389	eval-error:0.291139
[177]	train-error:0.305389	eval-error:0.291139
[178]	train-error:0.305389	eval-error:0.291139
[179]	train-error:0.305389	eval-error:0.291139
[180]	train-error:0.305389	eval-error:0.291139
[181]	train-error:0.305389	eval-error:0.291139
[182]	train-error:0.305389	eval-error:0.291139
[183]	train-error:0.305389	eval-error:0.291139
[184]	train-error:0.305389	eval-error:0.291139
[185]	train-error:0.305389	eval-error:0.291139
[186]	train-error:0.305389	eval-error:0.291139
[187]	train-error:0.305389	eval-error:0.291139
[188]	train-error:0.305389	eval-error:0.291139
[189]	train-error:0.305389	eval-error:0.291139
[190]	train-error:0.305389	eval-error:0.291139
[191]	train-error:0.305389	eval-error:0.291139
[192]	train-error:0.305389	eval-error:0.291139
[193]	train-error:0.305389	eval-error:0.291139
[194]	train-error:0.305389	eval-error:0.291139
[195]	train-e

In [125]:
preds = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)>0.5
1-np.mean(preds==Y[~msk])
#dataframe.ch.mean() # we can reach auc ~ 0.86, acc~0.8 better than random guess

0.21518987341772156

In [126]:
import operator

w = bst.get_score(importance_type='gain')
sorted_x = sorted(w.items(), key=operator.itemgetter(1), reverse=True)
sorted_x

[('新北', 7.793495),
 ('麵', 6.454516925130111),
 ('紫琳', 6.164954375),
 ('粥', 6.028923333333334),
 ('拉', 4.7606875),
 ('館', 4.708416363636363),
 ('汐止', 3.69036),
 ('中興新村', 3.402132),
 ('牛肉', 3.193034316860465),
 ('一中', 3.0747966120192323),
 ('廣宣', 2.9097242857142858),
 ('新竹', 2.8001438164864836),
 ('草屯', 2.6478592478260863),
 ('好', 2.55981),
 ('街', 2.3708444246610165),
 ('大', 2.36145),
 ('餐廳', 2.2186131),
 ('美食', 2.0509020833333333),
 ('中', 2.045679695),
 ('料理', 1.9044961600000005),
 ('食記', 1.8651987173076923),
 ('日式', 1.8085673333333332),
 ('商圈', 1.72792),
 ('台', 1.6034699523809524),
 ('南投', 1.5965023201849318),
 ('fw', 1.5771768571428573),
 ('咖啡', 1.5212510000000001),
 ('丼', 1.492201),
 ('平價', 1.4778529999999999),
 ('台北', 1.313556257894737),
 ('豆腐', 1.2920395000000002),
 ('東區', 1.2483824822916665),
 ('tapas', 1.189645),
 ('的', 0.9468423060606054),
 ('_', 0.7883335222222222),
 ('壹村', 0.781797),
 ('高雄', 0.6909728727272728),
 ('韓國', 0.5720715),
 ('店', 0.2989411166666667),
 ('北區', 0.2541873